In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [3]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  
    max_tokens=500,              
)

In [5]:
pdf_link = "os-sertoes.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,          
    chunk_overlap=20,         
    length_function=len,      
    add_start_index=True      
)

chunks = text_splitter.split_documents(pages)

In [7]:
vectordb = Chroma(embedding_function=embeddings)
vectordb.add_documents(chunks)
naive_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [8]:
rerank = CohereRerank(model="rerank-v3.5", top_n=3)

compressor_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,       
    base_retriever=naive_retriever,  
)

In [9]:
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado

Contexto: {context}
    
Pergunta: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [10]:
setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": compressor_retriever})  

output_parser = StrOutputParser()

compressor_retrieval_chain = setup_retrieval | rag_prompt | llm | output_parser

In [11]:
def answer_question(question: str):
    return compressor_retrieval_chain.invoke(question)

In [12]:
for index, question in enumerate(questions):
    resposta = answer_question(question)
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha apresenta uma visão contrastante sobre o ambiente natural do sertão nordestino. Por um lado, ele descreve o sertão como um vale fértil, um pomar vastíssimo sem dono. No entanto, ele também destaca os dias torturantes, a atmosfera asfixiadora, o solo empedrado, a nudez da flora e o espasmo assombrador da seca.\n\nEssa dualidade na descrição do sertão mostra que a natureza no sertão nordestino apresenta um jogo de antíteses, com terras férteis e áreas áridas convivendo lado a lado. Essa divisão especial na paisagem do sertão tem um papel importante na economia da região e influencia diretamente a vida dos habitantes.\n\nEuclides da Cunha aponta que a natureza não cria normalmente desertos, mas sim os combate e os repulsa. No entanto, no sertão nordestino, a seca e a aridez do solo criam um ambiente de ma